In [109]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
import matplotlib as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler


In [110]:
train_data = pd.read_csv("Train_data.csv")
train_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [111]:
# Create label encoders
label_encoders = {
    'protocol_type': LabelEncoder(),
    'service': LabelEncoder(),
    'flag': LabelEncoder(),
    'class': LabelEncoder()
}

# Fit and transform the columns
for column in ['protocol_type', 'service', 'flag', 'class']:
    train_data[column] = label_encoders[column].fit_transform(train_data[column])

# Review transformed data
train_data.head(20)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,19,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,2,41,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1,46,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1,22,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1,22,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
5,0,1,46,1,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,0
6,0,1,46,5,0,0,0,0,0,0,...,9,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
7,0,1,46,5,0,0,0,0,0,0,...,15,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,0
8,0,1,48,5,0,0,0,0,0,0,...,23,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
9,0,1,46,5,0,0,0,0,0,0,...,13,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,0


In [112]:
top_n_features = train_data.columns.drop('class') 

# Separate features (X) and target variable (y)
X = train_data[top_n_features]
y = train_data['class']

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [114]:
print('X_train:',np.shape(X_train))
print('y_train:',np.shape(y_train))
print('X_test:',np.shape(X_test))
print('y_test:',np.shape(y_test))

X_train: (17634, 41)
y_train: (17634,)
X_test: (7558, 41)
y_test: (7558,)


In [115]:
# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [116]:
# Define the TensorFlow model
model_tf = models.Sequential()
model_tf.add(layers.Dense(16, input_shape=(len(top_n_features),), activation='sigmoid'))

c:\Users\wasif\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [117]:
model_tf.add(Dense(1, activation='sigmoid'))

In [118]:
adam = keras.optimizers.Adam(learning_rate=0.001)

In [119]:
model_tf.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

In [120]:
model_tf.fit(X_train, y_train, epochs=100)

Epoch 1/100


552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 447us/step - accuracy: 0.7915 - loss: 0.4494
Epoch 2/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - accuracy: 0.9540 - loss: 0.1524
Epoch 3/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - accuracy: 0.9591 - loss: 0.1287
Epoch 4/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - accuracy: 0.9596 - loss: 0.1193
Epoch 5/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - accuracy: 0.9609 - loss: 0.1109
Epoch 6/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step - accuracy: 0.9660 - loss: 0.1028
Epoch 7/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 447us/step - accuracy: 0.9652 - loss: 0.1017
Epoch 8/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step - accuracy: 0.9669 - loss: 0.0949
Epoch 9/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - accuracy: 0.9705 - loss: 0.0861
Epoch 10/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - accuracy: 0.9730 - loss: 0.0757
Epoch 11/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - accuracy: 0.9753 - loss: 0.0723
Epoch 12/100
552/552 ━━━━━━━━━━

In [121]:
loss_and_metrics = model_tf.evaluate(X_test, y_test)
print(loss_and_metrics)
print('Loss = ',loss_and_metrics[0])
print(f'Test Accuracy: {loss_and_metrics[1] * 100:.2f}%')

237/237 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - accuracy: 0.9929 - loss: 0.0199
[0.021598374471068382, 0.9929875731468201]
Loss =  0.021598374471068382
Test Accuracy: 99.30%


In [122]:
# Evaluate your model on the test set
y_pred = model_tf.predict(X_test)
y_pred_binary = np.round(y_pred)  # Convert probabilities to binary predictions

# Calculate and print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)

237/237 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step
Confusion Matrix:
[[3499   17]
 [  36 4006]]


In [123]:
# Assuming your raw data does not include the target variable and follows the same order as your training data
raw_data = [0, "tcp", "http", "SF", 241, 1400, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33, 33, 0, 0, 0.03, 0.03, 1, 0, 0, 255, 255, 1, 0, 0, 0, 0, 0, 0, 0]

# Convert the raw data to a pandas DataFrame
column_names = top_n_features  # Make sure this includes all feature names except 'class'
test_data = pd.DataFrame([raw_data], columns=column_names)

# Apply the same preprocessing steps (label encoding and scaling)
for column in ['protocol_type', 'service', 'flag']:
    try:
        test_data[column] = label_encoders[column].transform(test_data[column])
    except KeyError as e:
        print(f"KeyError: {e}. The value might not have been seen during training.")

# Scale the features
test_data_scaled = scaler.transform(test_data)

# Make a prediction
prediction = model_tf.predict(test_data_scaled)
predicted_class = (prediction > 0.5).astype("int32")

# Print the predicted class
if predicted_class[0][0] == 1:
    print("Predicted class: Normal")
else:
    print("Predicted class: Anomaly")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted class: Normal
